In [17]:
import os
import gzip
import io
import json
from zipfile import ZipFile

In [18]:
def gunzip_bytes_obj(bytes_obj):
    in_ = io.BytesIO()
    in_.write(bytes_obj)
    in_.seek(0)
    with gzip.GzipFile(fileobj=in_, mode='rb') as fo:
        gunzipped_bytes_obj = fo.read()

    return gunzipped_bytes_obj.decode()

In [19]:
with ZipFile('RecipeTest.paprikarecipes', 'r') as zipObj:
    for filename in zipObj.namelist():
        print(filename)
        with zipObj.open(filename) as f:
            data = f.read()
            #data = unicode(data, errors='replace')
            data_json = gunzip_bytes_obj(data)
            d = json.loads(data_json)
            #print(d)

Air-Fryer Buffalo Chicken Drumsticks.paprikarecipe
{'cook_time': '45 minutes', 'photo': '370D4424-5E56-4EC2-959D-389186AD688C-9482-0000064CE43F7B92.jpg', 'servings': 'Serves 2', 'created': '2020-04-02 13:12:50', 'photo_hash': 'E1E69F9C888D62B4A4F489D6CF77FF0FB2B147160884F61E053BA6B15BB6A6BA', 'categories': ['Chicken'], 'name': 'Air-Fryer Buffalo Chicken Drumsticks', 'directions': 'Combine paprika, cayenne, salt, and pepper in bowl. Pat drumsticks dry with paper towels. Using metal skewer, poke 10 to 15 holes in skin of each drumstick. Rub with oil and sprinkle evenly with spice mixture.\n\nArrange drumsticks in air-fryer basket, spaced evenly apart, alternating ends. Place basket in air fryer and set temperature to 400 degrees. Cook until chicken is crisp and registers 195 degrees, 22 to 25 minutes, flipping and rotating chicken halfway through cooking. Transfer chicken to large plate, tent loosely with aluminum foil, and let rest for 5 minutes.\n\nMeanwhile, microwave hot sauce, butte

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
# comment out for now

for filename in os.listdir('/Users/jito/GitHub/joi.github.io/_recipes'):
#    if filename.endswith(".asm") or filename.endswith(".py"): 
        print(os.path.join('/Users/jito/GitHub/joi.github.io/_recipes', filename))
  #      continue
 #   else:
 #       continue

NameError: name 'os' is not defined

In [10]:
with open('RecipeTest.paprikarecipes', encoding='utf-8') as f:
  data = json.load(f.decode("utf-8","ignore"))
print (data)

AttributeError: '_io.TextIOWrapper' object has no attribute 'decode'

*afjd afsdf lkjasfdl;k jas*
asdf sd


asdf as